In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
import torchvision.transforms as transforms

In [2]:
# Download training data from open datasets.
# Guassian blur - how to do: RESEARCH
training_data = datasets.Flowers102(
    root="data",
    split="train",
    download=True,
    transform = transforms.Compose([transforms.Resize((64, 64)), transforms.ToTensor(), transforms.RandomHorizontalFlip(),
                                    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
                                    transforms.RandomVerticalFlip(), transforms.RandomRotation(45)])
    
)

# Download test data from open datasets.
test_data = datasets.Flowers102(
    root="data",
    split="test",
    download=True,
    transform = transforms.Compose([transforms.Resize((64, 64)), transforms.ToTensor(),
                                   transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))]),
)

# Download test data from open datasets.
test_data = datasets.Flowers102(
    root="data",
    split="val",
    download=True,
    transform = transforms.Compose([transforms.Resize((64, 64)), transforms.ToTensor(),
                                   transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))]),
)

100%|████████████████████████████████████████████████████████████████| 344862509/344862509 [08:02<00:00, 715113.73it/s]


Extracting data\flowers-102\102flowers.tgz to data\flowers-102


100%|████████████████████████████████████████████████████████████████████████████| 502/502 [00:00<00:00, 210175.74it/s]


100%|███████████████████████████████████████████████████████████████████████| 14989/14989 [00:00<00:00, 7491470.76it/s]


In [3]:
batch_size = 32
num_classes = 102
learning_rate = 0.001
num_epochs = 100

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

In [4]:
import torch.nn.functional as F
import os

class Net(nn.Module):
	#  Determine what layers and their order in CNN object 
    def __init__(self, num_classes):
        super(Net, self).__init__()
        self.conv_layer1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3)
        self.conv_layer2 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3)
        self.batch_layer1 = nn.BatchNorm2d(num_features=128) 
        self.max_pool1 = nn.MaxPool2d(kernel_size = 2, stride = 2)
        
        self.conv_layer3 = nn.Conv2d(in_channels=128, out_channels=512, kernel_size=3)
        self.conv_layer4 = nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3)
        self.batch_layer2 = nn.BatchNorm2d(num_features=512)
        self.max_pool2 = nn.MaxPool2d(kernel_size = 2, stride = 2)

        self.conv_layer5 = nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3)
        self.conv_layer6 = nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3)
        self.batch_layer3 = nn.BatchNorm2d(num_features=512)
        self.max_pool3 = nn.MaxPool2d(kernel_size = 1, stride = 2)
        
        self.conv_layer7 = nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3)
        self.conv_layer8 = nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3)
        self.batch_layer4 = nn.BatchNorm2d(num_features=512)
        self.max_pool4 = nn.MaxPool2d(kernel_size = 1, stride = 2)

        self.fc1 = nn.Linear(512, 128)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(128, num_classes)
    

    def forward(self, x):
        out = self.relu1(self.conv_layer1(x))
        out = self.relu1(self.conv_layer2(out))
        out = self.batch_layer1(out)
        out = self.max_pool1(out)
        
        out = self.relu1(self.conv_layer3(out))
        out = self.relu1(self.conv_layer4(out))
        out = self.batch_layer2(out)
        out = self.max_pool2(out)

        out = self.relu1(self.conv_layer5(out))
        out = self.relu1(self.conv_layer6(out))
        out = self.batch_layer3(out)
        out = self.max_pool3(out)
        
        out = self.relu1(self.conv_layer7(out))
        out = self.relu1(self.conv_layer8(out))
        out = self.batch_layer4(out)
        out = self.max_pool4(out)
                
        out = out.reshape(out.size(0), -1)
        
        out = self.fc1(out)
        out = self.relu1(out)
        out = self.fc2(out)
        return out

net = Net(num_classes)

def save_network(network, epoch_label):
    save_filename = 'net_%s.pth' % epoch_label
    save_path = os.path.join(save_filename)
    torch.save(network.state_dict(), save_path)
    
def testValidationData(network):
    net.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in test_dataloader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        return (100 * correct / total)

In [5]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [6]:
saveCounter = 0
minLoss = 10000
valAccurary = 0

# We use the pre-defined number of epochs to determine how many iterations to train the network on
for epoch in range(num_epochs):
	#Load in the data in batches using the train_loader object
    for i, (images, labels) in enumerate(train_dataloader):  
        # Move tensors to the configured device
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = net(images)
        loss = criterion(outputs, labels)
      
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if minLoss > loss:
        tempAcc = testValidationData(net)
        if tempAcc > valAccurary:
            valAccurary = tempAcc
            saveCounter += 1
            save_network(net, saveCounter)
            print("SAVE COMPLETE NEW ACC:", valAccurary)

    print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

SAVE COMPLETE NEW ACC: 2.9411764705882355
Epoch [1/100], Loss: 4.5743
SAVE COMPLETE NEW ACC: 4.803921568627451
Epoch [2/100], Loss: 4.4524
SAVE COMPLETE NEW ACC: 7.0588235294117645
Epoch [3/100], Loss: 4.0837
SAVE COMPLETE NEW ACC: 10.980392156862745
Epoch [4/100], Loss: 3.8519
SAVE COMPLETE NEW ACC: 13.137254901960784
Epoch [5/100], Loss: 3.7701
SAVE COMPLETE NEW ACC: 13.92156862745098
Epoch [6/100], Loss: 3.4215
Epoch [7/100], Loss: 3.1511
SAVE COMPLETE NEW ACC: 14.705882352941176
Epoch [8/100], Loss: 2.8954
SAVE COMPLETE NEW ACC: 16.764705882352942
Epoch [9/100], Loss: 3.1101
SAVE COMPLETE NEW ACC: 19.313725490196077
Epoch [10/100], Loss: 2.7463
Epoch [11/100], Loss: 3.2366
SAVE COMPLETE NEW ACC: 22.352941176470587
Epoch [12/100], Loss: 2.5171
Epoch [13/100], Loss: 2.8101
SAVE COMPLETE NEW ACC: 22.745098039215687
Epoch [14/100], Loss: 2.3526
SAVE COMPLETE NEW ACC: 24.019607843137255
Epoch [15/100], Loss: 2.7848
Epoch [16/100], Loss: 2.0866
SAVE COMPLETE NEW ACC: 24.705882352941178
E

In [7]:
net.eval()

with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_dataloader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    print('Accuracy of the network on the {} train images: {} %'.format(total, 100 * correct / total))

Accuracy of the network on the 1020 train images: 31.470588235294116 %


In [8]:
save_network(net, "FINALEPOCH")